# Table Header Detection

## Requirements
- Conda
- MongoDB instance
- PyMongo (will be installed by the notebook)

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} pymongo
# !conda install --yes --prefix {sys.prefix} premailer
# pip install premailer

import os
import json
import re
import pandas as pd
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from cssutils import parseStyle
from premailer import Premailer
import time
from dateutil.parser import parse
from datetime import datetime
import math

## Loading the seed data into mongoDB
- initial dataset [Wikipedia TabEL dataset](http://websail-fe.cs.northwestern.edu/TabEL/)
- dataset is lacking of some styling information
- we're crawling the wikipedia pages on our own
  - that should be feasible since we have to use labeled data only (both for training & testing)
  - we're taking the TabEL dataset pageID's as starting point, since we know that there should be at least one relational table

Each line of the TabEL dataset contains one JSON object representing a single table. However, the JSON objects are not contained within a JSON array. We need to wrap the single tables into an array first before we can parse the file as a whole.

In [ ]:
def wrapJSONObjectLineIntoTable(inputFilePath, outputFilePath):
    inputFile = open(inputFilePath, 'r')
    outputFile = open(outputFilePath, 'w')

    outputFile.write('[')

    previousLine = False
    for tableLineJsonObject in inputFile:
        if (previousLine):
            outputFile.write(previousLine + ',')
        previousLine = tableLineJsonObject
    if (previousLine):
        outputFile.write(previousLine)

    outputFile.write(']')

    inputFile.close()
    outputFile.close()

Check if TabEL dataset has been transformed into an array before. If not, we want to parse it now.

In [ ]:
inputFilePath = os.path.join('data', 'wikipedia_0_50000.json')
outputFilePath = os.path.join('data', 'wikipedia_0_50000_fixed.json')
if not os.path.isfile(outputFilePath):
    wrapJSONObjectLineIntoTable(inputFilePath, outputFilePath)

Parse JSON Array

In [ ]:
tabEL = pd.read_json(os.path.join('data', 'wikipedia_0_50000_fixed.json'))
tabEL.head()

Get 1000 unique page IDs and fetch the HTML content for it. (Update: We skip selecting only 1000 here, since we want a broader selection of pages/ tables.)

In [ ]:
uniquePageIDs = pd.DataFrame(tabEL['pgId'].unique(), columns=['pgId'])
print('Number of pages: ' + str(uniquePageIDs.shape[0]))

In [ ]:
def pickRandomSample():
    pageIDSample = uniquePageIDs.sample(n=1000)
    pageIDSample.reset_index(inplace=True)
    pageIDSample.drop(axis='columns', labels='index', inplace=True)

# pickRandomSample()

pageIDSample = uniquePageIDs
pageIDSample.head()

## Crawl the wikipedia pages and fetch all occurring tables
We use the pageID's from the TabEL dataset and crawl the wikipedia html. One page might include multiple tables. We only extract HTML tables with the class `wikitable`. The style from the CSS file gets parsed into inline style.

In [ ]:
BASE_URL = 'https://en.wikipedia.org'
wikipediaCSSFilePath = os.path.join('data', 'wikipedia.css')
cssFilePath = os.path.join('data', 'wikipedia.css')
cssFile = open(cssFilePath, 'r')
css = cssFile.read()
instance = Premailer(
    base_url=BASE_URL,
    exclude_pseudoclasses=False,
    include_star_selectors=True,
    disable_validation=True,
    css_text=css,
    allow_network=False,
    cssutils_logging_level='CRITICAL'
)

def crawl(tabEL):
    print(tabEL.name)
    payload = { 'curid': str(tabEL['pgId']) }
    html = requests.get(BASE_URL, params=payload).text
    htmlWithInlineCSS = instance.transform(str(html))
    return htmlWithInlineCSS

In [ ]:
client = MongoClient()
db = client.bob
pages = db.pages

def batchCrawl():
    pageIDSample['HTML'] = pageIDSample.apply(crawl, axis='columns')
    pageIDSample['HTML'] = pageIDSample['HTML'].str.replace('\n', '')
    pageIDSample['HTML'] = pageIDSample['HTML'].str.replace('\t', '')
    pages.insert_many(pageIDSample.to_dict('records'))
    # pageIDSample.to_json(os.path.join("data", "crawled.json"))
    client.close()

def sequenceCrawl():
    for index, row in pageIDSample.iterrows():
        inlineHTML = crawl(row)
        inlineHTML = inlineHTML.replace('\n', '')
        inlineHTML = inlineHTML.replace('\t', '')
        row['HTML'] = inlineHTML
        pages.insert_one(row.to_dict())
    client.close()

print('Begin time: ' + str(datetime.now()))
sequenceCrawl()

In [ ]:
client = MongoClient()
db = client.bob
pages = db.pages
pages.insert_many(pageIDSample.to_dict('records'))
# pageIDSample.to_json(os.path.join("data", "crawled.json"))
client.close()

In [ ]:
client = MongoClient()
db = client.bob
pages = db.pages
cursor = pages.find({})
pageIDSample = pd.DataFrame(list(cursor))
client.close()

In [ ]:
pageIDSample.head()

Now we extract the tables along with some metadata. For each row we assign an unique ID (the index of the row within the table) and a tag (whether the row includes `th-tags` only or is contained within a `thead`)

In [ ]:
HEADLINE_PATTERN = re.compile('(h|H)\d')
LABEL_CONTROLS = [
    {
        'label': 'Header',
        'color': 'light-blue'
    }, {
        'label': 'Data',
        'color': 'lime'
    }, {
        'label': 'Other',
        'color': 'orange'
    }
];

def extractPageTitle(soup):
    headlines = soup.select('h1')
    return headlines[0].text if len(headlines) > 0 else 'N/A'

def extractTableTitle(table):
    for sibling in table.previous_siblings:
        if (sibling is not None and sibling.name is not None and HEADLINE_PATTERN.match(sibling.name)):
            return sibling.text
    return 'N/A'

def addLabelControls(row, rowIndex, soup):
    labelControlTag = soup.new_tag(
        'th',
        attrs={
            'class': 'flex space-evenly'
        }
    )
    for labelControl in LABEL_CONTROLS:
        labelControlButton = soup.new_tag(
            'a',
            attrs={
                'class': 'labelButton waves-effect waves-light btn-small ' + labelControl['color'],
                'onClick': 'annotate(' + str(rowIndex) + ', "' + labelControl['label'] + '");',
            }
        )
        labelControlButton.string = labelControl['label']
        labelControlTag.append(labelControlButton)
    row.insert(0, labelControlTag)
    
def tagRow(row, rowIndex, soup, isHead=False):
    row['data-label'] = 'Header' if isHead else 'Data'
    row['data-row-index'] = rowIndex
    addLabelControls(row, rowIndex, soup)
    
def isHeaderRow(row):
    thTags = row.find_all('th', recursive=False)
    childCount = len(row.contents)
    return childCount == len(thTags) or row.parent.name == 'thead'

def tagRows(table, soup):
    rows = table.find_all('tr')
    annotations = []
    for rowIndex, row in enumerate(rows):
        isHeader = isHeaderRow(row)
        tagRow(row, rowIndex, soup, isHeader)  
        annotations.append('Header' if isHeader else 'Data')
    return annotations

def removeTableWidthLimitation(table):
    if not table.has_attr('style'):
        return
    tableStyle = parseStyle(table['style'])
    tableStyle['width'] = '100%'
    tableStyle['font-size'] = '100%'
    table['style'] = tableStyle.cssText
        
def extractTableInformation(table, pageID, tableIndex, pageTitle, soup):
    extractedInformation = {
        'pageID': pageID,
        'tableIndex': tableIndex,
        'pageTitle': pageTitle
    }
    extractedInformation['html'] = table.prettify()
    annotations = tagRows(table, soup)
    removeTableWidthLimitation(table)
    extractedInformation['taggedHtml'] = table.prettify()
    extractedInformation['annotations'] = annotations
    extractedInformation['tableTitle'] = extractTableTitle(table)
    return extractedInformation

def hasNestedTable(table):
    return len(table.select('table')) > 0

def extractTables(page):
    soup = BeautifulSoup(page['HTML'])
    pageTitle = extractPageTitle(soup)
    wikiTables = soup.select('table.wikitable')
    extractedTables = []
    for tableIndex, table in enumerate(wikiTables):
        if hasNestedTable(table):
            continue
        extractedTable = extractTableInformation(table, page['pgId'], tableIndex, pageTitle, soup)
        extractedTables.append(extractedTable)
    return extractedTables

In [ ]:
client = MongoClient()
db = client.bob
tables = db.tables
for extractedTables in pageIDSample.apply(extractTables, axis='columns').values:
    if len(extractedTables) > 0:
        tables.insert_many(extractedTables)
client.close()

The data can now get labeled using the provided [labeling tool](https://github.com/RichStone/web-tables-header-detection/tree/master/Labeling%20Tool). However, we may use the feature extraction to enhance the table selection process.

# Feature Extraction

In [ ]:
client = MongoClient()
db = client.bob
tables = db.tables
cursor = tables.find({})
tables = pd.DataFrame(list(cursor))
client.close()

In [ ]:
tables.head()

In [ ]:
SHORT_TEXT_THRESHOLD = 20
LONG_TEXT_THRESHOLD = 255

def isInt(value):
    try: 
        int(value)
        return True
    except ValueError:
        return False
    
def getRowSpan(cell):
    if cell.has_attr('rowspan') and isInt(cell['rowspan']):
        return int(cell['rowspan'])
    return 1
    
def getColSpan(cell):
    if cell.has_attr('colspan') and isInt(cell['colspan']):
        return int(cell['colspan'])
    return 1

def isMerged(cell):
    return (
        getColSpan(cell) > 1 or
        getRowSpan(cell) > 1
    )

def isCenterAligned(cell, style):
    return (
        (cell.has_attr('align') and cell['align'] == 'center') or
        (style is not None and 'text-align' in style and style['text-align'] == 'center')
    )

def isThOrInTHead(cell):
    row = cell.parent
    rowParent = row.parent
    return (
        cell.name == 'th' or
        rowParent.name == 'thead'
    )

def extractLayoutFeatures(cell, style):
    return {
        'isMerged': isMerged(cell),
        'isCenterAligned': isCenterAligned(cell, style),
        'isTHOrInTHead': isThOrInTHead(cell)
    }

def isBold(cell, style):
    return bool(
        style is not None and (
            style['font-weight'] == 'bold' or 
            style['font-style'] == 'bold'
        ) or
        cell.find('b') or
        cell.find('strong')
    )

def isItalic(cell, style):
    return bool(cell.find('i'))

def isUnderlined(cell, style):
    return bool(
        cell.find('u') or
        style is not None and (
            style['text-decoration'] == 'underline' or
            style['font-style'] == 'bold'
        )
    )

def isColored(cell, style):
    return (
        style is not None and (
            'background-color' in style or
            'color' in style
        )
    )

def extractStyleFeatures(cell, style):
    return {
        'isBold': isBold(cell, style),
        'isItalic': isItalic(cell, style),
        'isUnderlined': isUnderlined(cell, style)
    }

def getCellStyle(cell):
    return parseStyle(cell['style']) if cell.has_attr('style') else None

def getContentLength(cell):
    return len(re.sub('\s+',' ', cell.get_text()).split())

def isEmpty(cell):
    return getContentLength(cell) == 0

def isText(cell):
    return cell.get_text().isalpha()

def isNumeric(cell):
    return cell.get_text().isdigit()

def isDate(cell):
    try: 
        parse(cell.get_text(), fuzzy=False)
        return True
    except (ValueError, OverflowError):
        return False
    
def isShortText(cell):
    return getContentLength(cell) <= SHORT_TEXT_THRESHOLD

def isLongText(cell):
    return getContentLength(cell) > LONG_TEXT_THRESHOLD

def isTotal(cell):
    return cell.get_text().lower() == 'total'

def extractValueFeatures(cell):
    return {
        'isEmpty': isEmpty(cell),
        'isText': isText(cell),
        'isNumeric': isNumeric(cell),
        'isDate': isDate(cell),
        'isShortText': isShortText(cell),
        'isLongText': isLongText(cell),
        'isTotal': isTotal(cell)
    }

def mapDictBoolValuesToInt(dictionary):
    return { key: int(value) for key, value in dictionary.items() }

def applyColSpanFactor(dictionary, colSpan):
    return { key: value * colSpan for key, value in dictionary.items() }

def extractCellFeatures(cell, startRowIndex):
    cellStyle = getCellStyle(cell)
    boolCellFeatures = {
        **extractLayoutFeatures(cell, cellStyle),
        **extractStyleFeatures(cell, cellStyle),
        **extractValueFeatures(cell)
    }
    intCellFeatures = mapDictBoolValuesToInt(boolCellFeatures)
    # intCellFeatures = applyColSpanFactor(intCellFeatures, getColSpan(cell))
    cellFeatures = []
    rowSpan = getRowSpan(cell)
    for rowIndex in range(startRowIndex, startRowIndex + rowSpan):
        featureCopy = dict(intCellFeatures)
        featureCopy['row'] = rowIndex
        featureCopy['colCount'] = 1
        cellFeatures.append(featureCopy)
    return cellFeatures

def merge(featuresA, featuresB):
    return { k: featuresA.get(k, 0) + featuresB.get(k, 0) for k in set(featuresA) | set(featuresB) }

def stringifyDictKeys(dictionary):
    return { str(key): value for key, value in dictionary.items() }

def extractFeatures(table):
    soup = BeautifulSoup(table['html'])
    rows = soup.select('tr')
    rowsFeatures = {}
    for rowIndex, row in enumerate(rows):
        for cell in row.children:
            if type(cell) is not Tag:
                continue
            for rowFeatures in extractCellFeatures(cell, rowIndex):
                rowFeatureIndex = rowFeatures.pop('row')
                merged = merge(rowsFeatures.get(rowFeatureIndex, {}), rowFeatures)
                rowsFeatures[rowFeatureIndex] = merged
    rowsFeatures = stringifyDictKeys(rowsFeatures)
    return rowsFeatures

In [ ]:
tables['features'] = tables.apply(extractFeatures, axis='columns')
tables.head()

In [ ]:
client = MongoClient()
db = client.bob
tablesCollection = db.tables
tablesCollection.insert_many(tables.to_dict('records'))
client.close()

## Select Tables for Labeling
70% of all tables are considered 'simple'. We don't want to spend too much time labeling only simple tables. We wan't to have more interesting tables and are therefor making a thoughtful selection of tables.
Goal: 1000 Tables in total, where ideally 250 are randomly selected, 250 have no header, 250 have at least one merged cell and 250 tables that do have bold cells which are not located in the header.

In [ ]:
client = MongoClient()
db = client.bob
tables = db.tables
cursor = tables.find({})
tables = pd.DataFrame(list(cursor))
client.close()

In [ ]:
tables = tables.loc[lambda tables: tables['features'].apply(lambda features: len(features) > 1)]
print('Amount of tables with row count greater 1: ' + str(tables.shape[0]))

In [ ]:
def hasTableAHeader(tableFeatures):
    for rowIndex, rowFeatures in tableFeatures.items():  
        if (rowFeatures['isTHOrInTHead'] > 0):
            return True
    return False

tablesWithNoHeader = tables.loc[lambda tables: tables['features'].apply(lambda features: not hasTableAHeader(features))]
print('Amount of tables without a header: ' + str(tablesWithNoHeader.shape[0]))

In [ ]:
def hasBoldStyleOutsideHead(tableFeatures):
    for rowIndex, rowFeatures in tableFeatures.items():  
        if (rowFeatures['isBold'] > 0 and rowFeatures['isTHOrInTHead'] == 0):
            return True
    return False

tablesWithBoldStyleOutsideHead = tables.loc[lambda tables: tables['features'].apply(hasBoldStyleOutsideHead)]
print('Amount of tables with bold styles outside of header: ' + str(tablesWithBoldStyleOutsideHead.shape[0]))

In [ ]:
def hasMergedCellsOutsideHead(tableFeatures):
    for rowIndex, rowFeatures in tableFeatures.items():  
        if (rowFeatures['isMerged'] > 0 and rowFeatures['isTHOrInTHead'] == 0):
            return True
    return False

tablesWithMergedCellsOutsideHead = tables.loc[lambda tables: tables['features'].apply(lambda features: hasMergedCellsOutsideHead(features))]
print('Amount of tables with merged cells outside of header: ' + str(tablesWithMergedCellsOutsideHead.shape[0]))

In [ ]:
labelTables = tablesWithNoHeader.sample(n=250)

reducedTablesWithMergedCellsOutsideHead = tablesWithMergedCellsOutsideHead.loc[~tablesWithMergedCellsOutsideHead.index.isin(list(labelTables.index))]
labelTables = pd.concat([labelTables, reducedTablesWithMergedCellsOutsideHead.sample(n=250)])

reducedTablesWithBoldStyleOutsideHead = tablesWithBoldStyleOutsideHead.loc[~tablesWithBoldStyleOutsideHead.index.isin(list(labelTables.index))]
labelTables = pd.concat([labelTables, reducedTablesWithBoldStyleOutsideHead.sample(n=250)])

remainingTables = tables.loc[~tables.index.isin(list(labelTables.index))]
labelTables = pd.concat([labelTables, remainingTables.sample(n=250)])
labelTables = labelTables.sample(frac=1).reset_index(drop=True) #shuffle
labelTables.shape[0]

In [ ]:
labelTables1 = labelTables.head(500)
labelTables2 = labelTables.tail(500)

In [ ]:
client = MongoClient()
db = client.bob
labelTables1Collection = db.labelTables1
labelTables1Collection.insert_many(labelTables1.to_dict('records'))
labelTables2Collection = db.labelTables2
labelTables2Collection.insert_many(labelTables2.to_dict('records'))
client.close()

## Logarithmic Binning

In [ ]:
client = MongoClient()
db = client.bob
tables = db.tables
cursor = tables.find({})
tables = pd.DataFrame(list(cursor))
client.close()
tables.head()

In [ ]:
def calcA(c, r):
    if (c == 0):
        return 0
    if (c == r):
        return r
    if (c > r / 2.0):
        return math.floor(math.log2(r - c) + 1)
    return math.floor(math.log2(c) + 1)

def calcB(c, r):
    return math.floor(math.log2(r))

def isInSameBin(rowA, rowB, featureKey):
    return (
        calcB(rowA[featureKey], rowA['colCount']) == calcB(rowB[featureKey], rowB['colCount']) and 
        calcA(rowA[featureKey], rowA['colCount']) == calcA(rowB[featureKey], rowB['colCount'])
    )

def logBinTable(table):
    logBins = {}
    for rowIndex, row in table['features'].items():
        logBin = dict(row)
        colCount = logBin.pop('colCount')
        logBin = { 
            featureKey: { 
                'a': calcA(feature, colCount),
                'b': calcB(feature, colCount)
            } for featureKey, feature in logBin.items() 
        }
        logBins[rowIndex] = logBin
    return logBins

In [ ]:
tables['logBin'] = tables.apply(logBinTable, axis='columns')
tables.head()

In [ ]:
client = MongoClient()
db = client.bob
tablesCollection = db.tables
tablesCollection.insert_many(tables.to_dict('records'))
client.close()